In [1]:
from collections import defaultdict
import json
import statistics

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib notebook

In [2]:
years_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/years.csv")

In [3]:
team_years_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/team_years.csv")

In [4]:
events_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/events.csv")

In [5]:
matches_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/matches.csv")

In [6]:
year_matches_dict = {k: v for k, v in matches_df.groupby(["year"])}
year_teams_dict = {k: v for k, v in team_years_df.groupby(["year"])}

In [7]:
year_matches_dict[2017].iloc[0]

key                    2017abca_f1m1
year                            2017
event                       2017abca
comp_level                         f
set_number                         1
match_number                       1
status                     Completed
red                   1482,2122,4334
red_elo_sum                   4917.0
red_opr_sum                   315.48
red_ils_1_sum                    NaN
red_ils_2_sum                    NaN
blue                   244,5015,4191
blue_elo_sum                  4683.0
blue_opr_sum                  288.69
blue_ils_1_sum                   NaN
blue_ils_2_sum                   NaN
winner                           red
elo_winner                       red
elo_win_prob                  0.7936
opr_winner                       red
opr_win_prob                  0.6334
mix_winner                       red
mix_win_prob                  0.7135
red_rp_1_prob                    NaN
red_rp_2_prob                    NaN
blue_rp_1_prob                   NaN
b

In [60]:
elos = defaultdict(dict)
auto_elos = defaultdict(dict)
teleop_elos = defaultdict(dict)
endgame_elos = defaultdict(dict)
all_elos = defaultdict(dict)

for year in range(2014, 2016):
    for _, team_year in year_teams_dict[year].iterrows():
        team_num = team_year.team
        elos[year][team_num] = team_year.elo_start
        all_elos[year][team_num] = team_year.elo_start

In [51]:
def get_stats(year):
    week_one = list(events_df[(events_df.year == year) & (events_df.week == 1)]["key"])
    autos, teleops, endgames, scores = [], [], [], []
    for _, match in matches_df[(matches_df.year == year)].iterrows():
        if match.event in week_one:
            if match.red_auto >= 0 and match.blue_auto >= 0:
                autos.extend([match.red_auto, match.blue_auto])
            if match.red_teleop >= 0 and match.blue_teleop >= 0:
                teleops.extend([match.red_score - match.red_auto - match.red_endgame, match.blue_score - match.blue_auto - match.red_auto])
            if match.red_endgame >= 0 and match.blue_endgame >= 0:
                endgames.extend([match.red_endgame, match.blue_endgame])
            if match.red_score >= 0 and match.blue_score >= 0:
                scores.extend([match.red_score, match.blue_score])

    auto_mean, auto_sd = sum(autos) / len(autos), statistics.pstdev(autos)
    teleop_mean, teleop_sd = sum(teleops) / len(teleops), statistics.pstdev(teleops)
    endgame_mean, endgame_sd = sum(endgames) / len(endgames), statistics.pstdev(endgames)
    score_mean, score_sd = sum(scores) / len(scores), statistics.pstdev(scores)

    return auto_mean, auto_sd, teleop_mean, teleop_sd, endgame_mean, endgame_sd, score_mean, score_sd

year_stats = {year: get_stats(year) for year in range(2016, 2021)}

In [68]:
global_power = 0.90
global_factor = 0.25

def score_pred_func(red_score, blue_score, pred_margin):
    red_win_prob = 1 / (10 ** (250 * (-pred_margin) / 400) + 1)
    
    if red_score > blue_score:
        return red_win_prob >= 0.5, (1 - red_win_prob) ** 2 
    elif red_score < blue_score:
        return red_win_prob <= 0.5, (red_win_prob) ** 2
    
    return 0, (0.5 - red_win_prob) ** 2

def nl(x, power):
    if x == 0:
        return x
    return x / abs(x) * abs(x) ** power


def elo_sum(x, factor):
    x = list(x)
    return sum(x) + factor * max(x) - factor * min(x)

def update_elo(elo, update, elos, factor):
    mult = 1
    if elo == max(elos):
        mult += factor
    elif elo == min(elos):
        mult -= factor
    return round(elo + mult * update, 2)

total_acc_new, total_mse_new = 0, 0
total_acc_old, total_mse_old = 0, 0
total_acc_mix, total_mse_mix = 0, 0
for year in range(2016, 2021):
    count = 0
    acc_new, mse_new = 0, 0
    acc_old, mse_old = 0, 0
    acc_mix, mse_mix = 0, 0
    
    years_row = years_df[years_df.year == year].iloc[0]
    
    elo_acc, elo_mse = years_row.elo_acc, years_row.elo_mse
    auto_mean, auto_sd, teleop_mean, teleop_sd, endgame_mean, endgame_sd, score_mean, score_sd = year_stats[year]
    
    year_matches_df = year_matches_dict[year].sort_values(by=["time"])
    year_teams_df = year_teams_dict[year]
    
    for _, team_year in year_teams_df.iterrows():
        team_num = team_year.team
        
        for (elo_dict, source_elo_dict) in [(elos, elos), (auto_elos, auto_elos), (teleop_elos, all_elos), (endgame_elos, all_elos)]:
            elo_1yr = source_elo_dict[year - 1].get(team_num, elos[year - 1].get(team_num, 1500))
            elo_2yr = source_elo_dict[year - 2].get(team_num, elos[year - 2].get(team_num, 1500))
            elo_dict[year][team_num] = 0.56 * elo_1yr + 0.24 * elo_2yr + 0.20 * 1450
    
    for _, match in year_matches_df.iterrows():
        if not (match.red_auto >= 0 and match.red_teleop >= 0 and match.red_endgame >= 0 and match.red_score >= 0):
            continue
            
        red_teams = [int(x) for x in match.red.split(",")]
        blue_teams = [int(x) for x in match.blue.split(",")]
        
        red_elo_sum = elo_sum([elos[year][x] for x in red_teams], 0)
        blue_elo_sum = elo_sum([elos[year][x] for x in blue_teams], 0)
        pred_margin1 = (red_elo_sum - blue_elo_sum) / 250
        acc, mse = score_pred_func(match.red_score, match.blue_score, pred_margin1)
        acc_old += acc
        mse_old += mse
        
        red_auto_pred = auto_mean + auto_sd * nl((elo_sum([auto_elos[year][x] for x in red_teams], global_factor) - 4500) / 250, global_power)
        blue_auto_pred = auto_mean + auto_sd * nl((elo_sum([auto_elos[year][x] for x in blue_teams], global_factor) - 4500) / 250, global_power)
        red_endgame_pred = endgame_mean + endgame_sd * nl((elo_sum([endgame_elos[year][x] for x in red_teams], global_factor) - 4500) / 250, global_power)
        blue_endgame_pred = endgame_mean + endgame_sd * nl((elo_sum([endgame_elos[year][x] for x in blue_teams], global_factor) - 4500) / 250, global_power)
        red_teleop = nl(elo_sum([teleop_elos[year][x] for x in red_teams], global_factor) / 250, global_power)
        blue_teleop = nl(elo_sum([teleop_elos[year][x] for x in blue_teams], global_factor) / 250, global_power)
        teleop_margin = teleop_sd * (red_teleop - blue_teleop)
        pred_margin2 = ((red_auto_pred - blue_auto_pred) + teleop_margin + (red_endgame_pred - blue_endgame_pred)) / score_sd
        acc, mse = score_pred_func(match.red_score, match.blue_score, pred_margin2)
        acc_new += acc
        mse_new += mse
        
        acc, mse = score_pred_func(match.red_score, match.blue_score, (pred_margin1 + pred_margin2) / 2)
        acc_mix += acc
        mse_mix += mse
        
        count += 1
        
        k = 4 if match.playoff else 12
        
        for (mean, sd, elo_dict, teams, get_func, power, factor) in [
            (auto_mean, auto_sd, auto_elos, red_teams, lambda m: m.red_auto, global_power, global_factor),
            (auto_mean, auto_sd, auto_elos, blue_teams, lambda m: m.blue_auto, global_power, global_factor),
            (endgame_mean, endgame_sd, endgame_elos, red_teams, lambda m: m.red_endgame, global_power, global_factor),
            (endgame_mean, endgame_sd, endgame_elos, blue_teams, lambda m: m.blue_endgame, global_power, global_factor),
        ]:
            temp_elos = [elo_dict[year][x] for x in teams]
            temp_elo_sum = elo_sum([elo_dict[year][x] for x in teams], factor)
            score_pred = mean / sd + nl((temp_elo_sum - 4500) / 250, power)
            score = get_func(match) / sd
            for x, temp_elo in zip(teams, temp_elos):
                elo_dict[year][x] = update_elo(elo_dict[year][x], k * (score - score_pred), temp_elos, factor)
        
        for (sd, elo_dict, red_get_func, blue_get_func, power, factor) in [
            (teleop_sd, teleop_elos, lambda m: m.red_score - m.red_auto - m.red_endgame, lambda m: m.blue_score - m.blue_auto - m.blue_endgame, global_power, global_factor),
            (score_sd, elos, lambda m: m.red_score, lambda m: m.blue_score, 1, 0),
        ]:
            red_elos = [elo_dict[year][x] for x in red_teams]
            red_elo_sum = elo_sum([elo_dict[year][x] for x in red_teams], factor)
            blue_elos = [elo_dict[year][x] for x in blue_teams]
            blue_elo_sum = elo_sum([elo_dict[year][x] for x in blue_teams], factor)
            win_margin = (red_get_func(match) - blue_get_func(match)) / sd
            pred_win_margin = nl(red_elo_sum / 250, power) - nl(blue_elo_sum / 250, power)
            for x, temp_elo in zip(red_teams, red_elos):
                elo_dict[year][x] = update_elo(elo_dict[year][x], k * (win_margin - pred_win_margin), red_elos, factor)
            for x, temp_elo in zip(blue_teams, blue_elos):
                elo_dict[year][x] = update_elo(elo_dict[year][x], -k * (win_margin - pred_win_margin), blue_elos, factor)
    
    all_elos[year] = {
        k: round(
        1500 + 
        (auto_sd * (auto_elos[year][k] - 1500) + teleop_sd * (teleop_elos[year][k] - 1500) + endgame_sd * (endgame_elos[year][k] - 1500)) / 
        (auto_sd + teleop_sd + endgame_sd), 2) for k in elos[year].keys()
    }
    
    total_acc_new += acc_new / count
    total_mse_new += mse_new / count
    total_acc_old += acc_old / count
    total_mse_old += mse_old / count
    total_acc_mix += acc_mix / count
    total_mse_mix += mse_mix / count

    print(year, "\t", round(acc_new / count, 4), round(mse_new / count, 4), "\t", round(acc_old / count, 4), round(mse_old / count , 4), "\t", round(acc_mix / count, 4), round(mse_mix / count, 4))
    
print()
print("Avg", "\t", round(total_acc_new / 5, 4), round(total_mse_new / 5, 4), "\t", round(total_acc_old / 5, 4), round(total_mse_old / 5 , 4), "\t", round(total_acc_mix / 5, 4), round(total_mse_mix / 5, 4))

2016 	 0.6995 0.189 	 0.6952 0.1909 	 0.6988 0.1893
2017 	 0.6542 0.2088 	 0.6406 0.2113 	 0.6534 0.208
2018 	 0.745 0.1728 	 0.7334 0.1757 	 0.7414 0.1733
2019 	 0.7066 0.1815 	 0.6983 0.1847 	 0.7054 0.1823
2020 	 0.7075 0.1911 	 0.699 0.1953 	 0.704 0.1923

Avg 	 0.7026 0.1886 	 0.6933 0.1916 	 0.7006 0.189


In [67]:
year = 2020

print(list(sorted(auto_elos[year].items(), key=lambda x: -x[1]))[:10])
print(list(sorted(teleop_elos[year].items(), key=lambda x: -x[1]))[:10])
print(list(sorted(endgame_elos[year].items(), key=lambda x: -x[1]))[:10])

_, auto_sd, _, teleop_sd, _, endgame_sd, _, _ = year_stats[year]
elos_temp = {k: round(1500 + (auto_sd * (auto_elos[year][k] - 1500) + teleop_sd * (teleop_elos[year][k] - 1500) + endgame_sd * (endgame_elos[year][k] - 1500)) / (auto_sd + teleop_sd + endgame_sd), 2) for k in auto_elos[year].keys()}
print(list(sorted(elos_temp.items(), key=lambda x: -x[1]))[:10])

print(list(sorted(elos[year].items(), key=lambda x: -x[1]))[:10])

[(1678, 1827.17), (5172, 1751.33), (6560, 1743.64), (973, 1730.23), (1619, 1729.34), (2337, 1717.34), (67, 1707.57), (694, 1703.55), (4020, 1703.55), (1690, 1698.07)]
[(1690, 1881.55), (973, 1873.42), (3683, 1847.45), (148, 1843.44), (987, 1840.63), (4635, 1839.71), (2910, 1837.25), (1678, 1832.98), (16, 1824.91), (118, 1811.96)]
[(5436, 1710.62), (1325, 1706.0), (148, 1699.92), (2052, 1693.05), (1690, 1690.05), (5199, 1688.86), (2338, 1684.63), (1619, 1678.51), (4539, 1677.56), (3824, 1677.29)]
[(1690, 1777.53), (973, 1765.74), (1678, 1761.53), (148, 1758.89), (5172, 1740.78), (987, 1737.56), (2910, 1732.94), (694, 1728.28), (67, 1727.01), (1619, 1724.45)]
[(148, 1898.71), (973, 1865.19), (1690, 1856.3), (1678, 1854.87), (2910, 1835.22), (5172, 1830.87), (987, 1819.37), (2168, 1817.24), (694, 1808.04), (118, 1804.12)]
